## PLN1 - Procesamiento del Lenguaje Natural I ##

Nombre: José Aviani

Código: a2103

### Desafío 1 ###

Instalamos las librarías necesarias:

In [1]:
%pip install numpy scikit-learn pandas

Note: you may need to restart the kernel to use updated packages.


Hacemos los imports necesarios:

In [2]:
import random
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups

In [3]:
random.seed(84)

Cargamos el dataset:

In [4]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

Creamos el vectorizador, hacemos un fit y transform sobre los datos de train y transform sobre los datos de test:

In [5]:
tfidfvect = TfidfVectorizer()

X_train_text = newsgroups_train.data
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

X_test_text  = newsgroups_test.data
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

**Punto 1**

Generamos 5 números al azar con las posiciones de los documentos a analizar:

In [6]:
numeros_rndm = sorted(random.sample(range(0, X_train.shape[0]), 5))
numeros_rndm

[71, 611, 4646, 8013, 8547]

Para asegurarnos que no cambien esos números (cosa que no debería suceder setando la semilla) inicializamos una lista con esos valores:

In [7]:
documentos_idx = [71, 611, 4646, 8013, 8547]

Hacemos el análisis y comparación sobre los 5 documentos:

In [20]:
def _analizar_documentos_similares(documentos_idx):
  # Recorremos los 5 documentos seleccionados
  for idx_1, documento_idx_1 in enumerate(documentos_idx):
    print("-" * 120)
    print("-" * 120)
    print(f"Documento número {idx_1 + 1}")
    print("-" * 18)
    print(f"Índice: {documento_idx_1}")
    print(f"Clase: {newsgroups_train.target_names[y_train[documento_idx_1]]}")
    print("Contenido:")
    print("-" * 100)
    print(newsgroups_train.data[documento_idx_1])
    print("-" * 100)
    # Calculamos la similaridad coseno con todos los documentos de train
    cossim = cosine_similarity(X_train[documento_idx_1], X_train)[0]
    # Ordenamos por similitud, salteamos el primero (que es el mismo documento) y tomamos los siguientes 5 para analizar
    # (es decir, los 5 más similares)
    for idx_2, documento_idx_2 in enumerate(np.argsort(cossim)[::-1][1:6]):
      print("     " + "-" * 80)
      print(f"     Comparación {idx_2 + 1}")
      print("     " + "-" * 13)
      print(f"     Índice: {documento_idx_2}")
      print(f"     Similaradidad coseno: {cossim[documento_idx_2]}")
      print(f"     Clase: {newsgroups_train.target_names[y_train[documento_idx_2]]}")
      print("     Contenido:")
      print("     " + "-" * 60)
      print(newsgroups_train.data[documento_idx_2])
      print("     " + "-" * 80)
    print("-" * 120)
    print("-" * 120)
    print("\n" * 3)


_analizar_documentos_similares(documentos_idx)

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
Documento número 1
------------------
Índice: 71
Clase: rec.autos
Contenido:
----------------------------------------------------------------------------------------------------

Yo! Watch the attributions--I didn't say that!

Again, this isn't an appropriate forum for discussions on whether you
should shoot someone for property damage/vandalism/theft, but every
responsible gun owner realizes that there are limits, and the punishment
must fit the crime. I mean, think about it--is a (really) harmless
prank worth killing over?

As I said, the situation described (punks setting off alarms and
taunting people to come out) could turn very ugly very quickly, and
it is worth being prepared when your life is potentially on the line.

				James
------------

**Análisis**

<br />

Documento 1:

Está clasificado como "rec.autos", pero su contenido tiene que ver casi exclusivamente con armas, defensa propia y violencia: tiene más sentido que su clasificación sea "talk.politics.guns".

Los 5 documentos más similares pertenecen casi todos a "talk.politics.guns" (debates sobre control de armas, crimen, asesinatos, responsabilidad de las leyes) y uno a talk.politics.mideast con descripciones de masacres y violencia extrema.

Por contenido, la similaridad tiene sentido: todos los textos comparten vocabulario y temática de armas, violencia y conflictos graves.

Por clasificación, se ve una clara desalineación, pero esto se debe a que el documento está mal clasificado..

La medida de similaridad está capturando bien la proximidad temática.

<br />

Documento 2:

Consiste casi sólo en una cita bíblica sobre pensar en lo verdadero, honorable, justo, puro, etc. y su clasificación es "talk.religion.misc".

Sus vecinos más similares incluyen principalmente textos clasificados como "alt.atheism" con discusiones sobre verdad, promesas, evidencias y qué es "Truth", y también un documento de "sci.space" que habla de problemas con detectores en el espacio. 

Cuando la clase es "alt.atheism" o "talk.religion.misc", la similaridad tiene sentido: los textos comparten vocabulario y contexto de discusión religiosa / filosófica (verdad, creencias, lo que es correcto, etc.).

El vecino de "sci.space" muestra una limitación: el documento original es tan corto y genérico que unas pocas palabras comunes alcanzan para producir una similitud, aunque temáticamente el contenido no sea religioso.

La similaridad coseno refleja bien la cercanía temática entre textos religiosos y ateos (más allá de la etiqueta de clase), pero en documentos muy cortos aparecen inconsistencias: aparece algún vecino de otra clase cuya similaridad es más de vocabulario que temática.

<br />

Documento 3:

El texto es breve y personal: es un reproche íntimo o reflexión sobre autoestima y capacidad de cuidar a otros. Su clasificación es "alt.atheism".

Entre los documentos más similares aparece, por ejemplo, uno de "talk.politics.mideast" donde se discute sobre derechos humanos, Palestina, si es legítimo matar, etc.

A nivel de palabras, la similaridad coseno es entendible: aparecen términos genéricos como people, good, rights, care, debates morales, etc., y eso basta para la similitud.

A nivel temático y de clase, la similaridad es mucho menos clara: el documento base es casi una carta personal, mientras que los vecinos son debates políticos y sobre derechos humanos, con etiquetas "talk.politics".

La similitud coseno parece capturar sobre todo vocabulario moral genérico, pero no tanto el tipo de discurso (personal y político), y por eso las clases y los temas no concuerdan.

<br />

Documento 4:

El texto es un post de autos: explica cómo calcular el pago del préstamo del auto actual, cómo usar el valor de entrega como parte de pago, financiación y compra/venta de autos. Su clase es "rec.autos". 

Los documentos más similares pertenecen también a "rec.autos" y hablan de experiencias con autos. 

En este caso, la similaridad coseno tiene mucho sentido tanto por contenido como por la clasificación: todos los textos tratan de propiedad, confiabilidad, costos y decisiones económicas alrededor del auto, y las clases de los vecinos coinciden con "rec.autos".

<br />

Documento 5:

El texto habla de la situación en Gaza: restricciones, ocupación, uso de la fuerza, vínculos entre terroristas y población general, etc. Es claramente un texto de conflicto político y derechos humanos en Medio Oriente. Su clasificación es "talk.politics.mideast".

Entre sus vecinos más similares aparece, por ejemplo, un documento de la clase "talk.politics.mideast" que describe en detalle pogroms, con alusiones a masacres, genocidio y violaciones de derechos humanos.

Otros vecinos vienen de grupos políticos cercanos ("talk.politics.misc", "soc.culture") pero siguen tratando conflictos étnicos, violencia estatal o represión.

Por contenido, la similaridad tiene sentido: todos los textos comparten vocabulario y contenido de denuncia política

Por clase, aunque no siempre coincida el newsgroup exacto, todos pertenecen a ámbitos políticos o geopolíticos, por lo que la diferencia de clase es menor que en otros documentos.

La similitud coseno es coherente: los vecinos son de la misma familia temática (conflictos violentos y derechos humanos), y las clases, aunque no siempre idénticas, son todas del mismo bloque político.

<br />

Conclusiones:

La similitud coseno capta bien el tema real del texto. Cuando un documento está mal clasificado, sus vecinos son de los grupos cuyo contenido sí coincide.

Cuando las clases no coinciden, muchas veces igual tiene sentido.

Los peores emparejamientos aparecen con textos muy cortos o genéricos. Ahí la similitud se basa en pocas palabras comunes y puede traer documentos de clases temáticas muy distintas, aunque compartan algo de vocabulario.

Con textos largos y bien enfocados, la similitud y la clase suelen alinearse.

La etiqueta del newsgroup no siempre refleja toda la temática real. La similitud coseno tiende a agrupar por contenido efectivo, y cuando no coincide con la clase, muchas veces el problema está en la clasificación original o en que el post se desvió de tema.

**Punto 2**

Definimos nuestro modelo:

In [9]:
class ModeloClasificacionPrototipo:

  def __init__(self):
    self.tfidfvect = TfidfVectorizer()
    self.X_train = None
    self.y_train = None


  def train(self, data, target):
    if data is None:
      raise ValueError("El parámetro 'data' no puede ser None.")
    if not isinstance(data, list):
      raise TypeError("El parámetro 'data' debe ser una lista.")
    if len(data) == 0:
      raise ValueError("El parámetro 'data' no puede estar vacío.")
    if target is None:
      raise ValueError("El parámetro 'target' no puede ser None.")
    if not isinstance(target, np.ndarray):
      raise TypeError(f"El parámetro 'target' debe ser un numpy.ndarray.")
    if len(target) == 0:
      raise ValueError("El parámetro 'target' no puede estar vacío.")
    if len(data) != len(target):
      raise ValueError(f"La cantidad de elementos de 'data' y 'target' debe ser la misma.")

    self.X_train = self.tfidfvect.fit_transform(data)
    self.y_train = target


  def predict(self, X):
    if X is None:
      raise ValueError("El parámetro 'X' no puede ser None.")
    if isinstance(X, list):
      if len(X) == 0:
        raise TypeError("Si el parámetro 'X' es una lista, no puede estar vacía.")
    elif isinstance(X, str):
      if X.strip() == "":
        raise ValueError("Si el parámetro 'X' es un string, no puede ser vacío.")
    else:
      raise TypeError("El parámetro 'X' debe ser una lista o un string.")
    if self.X_train is None or self.y_train is None:
      raise ValueError("Error: el modelo aún no fue entrenado.")
    
    if isinstance(X, str):
      X = [X]

    X_vectorized = self.tfidfvect.transform(X)
    predictions = []
    for x in X_vectorized:
      cossim = cosine_similarity(x, self.X_train)[::-1][0]
      sim_idx = np.argsort(cossim)[::-1][0]

      predictions.append(self.y_train[sim_idx])

    return predictions

Creamos nuestro modelo, lo "entrenamos" y evaluamos:

In [10]:
modeloClasificacionPrototipo = ModeloClasificacionPrototipo()
modeloClasificacionPrototipo.train(newsgroups_train.data, newsgroups_train.target)

In [11]:
predicciones = modeloClasificacionPrototipo.predict(newsgroups_test.data)

F1 Score de nuestro modelo de clasificación por prototipos (tipo zero-shot):

In [12]:
f1_score(y_test, predicciones, average='macro')

0.5036472862947965

Este punto consistía en implementar un modelo de clasificación según el criterio aplicado en el punto 1, clasificando el documento con la misma clase que el documento de mayor similitud coseno.

Con un F1 score de 0.5 evidentemente no es un buen modelo, aunque podemos decir que, teniendo en cuenta lo básico y fácil de implementar que es, no está tan mal.

**Punto 3**

Hacemos un grid search de hiperparámetros para los modelos Naïve Bayes Multinomial y ComplementNB con F1-score utilizando los datos de test. Al final mostramos el mejor modelo:

In [13]:
def _modelo_naive_bayes_grid_search(X_train_text, y_train, X_test_text, y_test, stop_words_list, min_df_list, max_df_list, alpha_list):

  # No cambiamos el hiperparametro ngram_range
  ngram_range=(1,1)

  # Armamos las combinaciones de vectorizadores
  vectorizers = []
  for stop_words in stop_words_list:
    for min_df in min_df_list:
      for max_df in max_df_list:
        for b in [True, False]:
            
          # CountVectorizer
          vect = CountVectorizer(
            ngram_range=ngram_range,
            lowercase=True,
            stop_words=stop_words,
            min_df=min_df,
            max_df=max_df,
            binary=b
          )
          vectorizers.append(("count", vect))
     
          for sublinear_tf in [False, True]:
            # TfidfVectorizer
            vect = TfidfVectorizer(
              ngram_range=ngram_range,
              lowercase=True,
              stop_words=stop_words,
              min_df=min_df,
              max_df=max_df,
              use_idf=b,
              sublinear_tf=sublinear_tf
            )
            vectorizers.append(("tfidf", vect))


  # Armamos las combinaciones de modelos
  models = []
  for alpha in alpha_list:
    models.append(("MultinomialNB", MultinomialNB(alpha=alpha)))
    models.append(("ComplementNB", ComplementNB(alpha=alpha)))


  # Evaluamos todas las combinaciones de vectorizadores y modelos
  resultados = []
  for vect_name, vect in vectorizers:
    Xtr = vect.fit_transform(X_train_text)
    Xte = vect.transform(X_test_text)

    for model_name, model in models:
      # Entrenamos, predecimos y evaluamos
      model.fit(Xtr, y_train)
      y_pred = model.predict(Xte)
      f1 = f1_score(y_test, y_pred, average='macro')
      # Agregamos los resultados
      resultados.append({
        "vectorizer": vect_name,
        "stop_words": vect.stop_words,
        "min_df": vect.min_df,
        "max_df": vect.max_df,
        "binary": getattr(vect, "binary", None),
        "use_idf": getattr(vect, "use_idf", None),
        "sublinear_tf": getattr(vect, "sublinear_tf", None),
        "model": model_name,
        "alpha": getattr(model, "alpha", None),
        "f1_macro_test": f1
      })
  # Ordenamos los resultados por F1-macro
  df_res = pd.DataFrame(resultados).sort_values("f1_macro_test", ascending=False).reset_index(drop=True)


  # Mostramos las mejores combinaciones
  top_combinaciones = 25
  print(f"Top {top_combinaciones} combinaciones por F1-macro:")
  display(df_res.head(top_combinaciones))


  # Mejor modelo
  best = df_res.iloc[0]
  print("\nMejor configuración encontrada:")
  for k, v in best.items():
    print(f"  {k}: {v}")



# Espacio de búsqueda (sin cambiar ngram_range):
#    - Tfidf: variar stop_words, min_df, max_df, use_idf, sublinear_tf
#    - Count: variar stop_words, min_df, max_df, binary
stop_words_list = [None, 'english']
min_df_list = [1, 2, 5]
max_df_list = [1.0, 0.9, 0.8]
alpha_list = [0.1, 0.5, 1.0, 2.0, 5.0]

_modelo_naive_bayes_grid_search(X_train_text, y_train, X_test_text, y_test, stop_words_list, min_df_list, max_df_list, alpha_list)

Top 25 combinaciones por F1-macro:


,vectorizer,stop_words,min_df,max_df,binary,use_idf,sublinear_tf,model,alpha,f1_macro_test
0,tfidf,english,1,1.0,False,False,False,ComplementNB,0.1,0.700237
1,tfidf,english,1,0.9,False,False,False,ComplementNB,0.1,0.700237
2,tfidf,english,1,0.8,False,False,False,ComplementNB,0.1,0.700237
3,tfidf,None,1,1.0,False,False,True,ComplementNB,0.1,0.699825
4,tfidf,None,1,0.9,False,False,True,ComplementNB,0.1,0.699825
5,tfidf,None,1,0.8,False,False,True,ComplementNB,0.1,0.699818
6,tfidf,None,1,0.8,False,False,False,ComplementNB,0.1,0.699564
7,tfidf,None,1,0.9,False,False,False,ComplementNB,0.1,0.698741
8,tfidf,None,1,1.0,False,False,False,ComplementNB,0.1,0.698741
9,tfidf,english,1,0.9,False,False,True,ComplementNB,0.1,0.698650



Mejor configuración encontrada:
  vectorizer: tfidf
  stop_words: english
  min_df: 1
  max_df: 1.0
  binary: False
  use_idf: False
  sublinear_tf: False
  model: ComplementNB
  alpha: 0.1
  f1_macro_test: 0.7002367312899178


Conclusiones:

Modelo: El modelo de las 25 mejores combinaciones es ComplementNB. Para este problema, ComplementNB se adapta mejor que MultinomialNB, algo esperable en textos con clases desbalanceadas.

Vecorizador: El vectorizador de las 25 mejores combinaciones es tfidf. Para este dataset, ponderar términos por frecuencia relativa en el corpus ayuda más que usar simples conteos.

Cambios en stop_words (None, english), min_df (1, 2) y max_df (1.0, 0.9, 0.8) producen variaciones pequeñas en el F1-macro (todas las entradas están muy cerca de 0.70). Eso muestra que el rendimiento se estabiliza alrededor de 0.69–0.70, y que una vez elegido correctamente el tipo de modelo (ComplementNB + TF-IDF), los ajustes finos del vectorizador mejoran poco.

La mejor combinación tiene use_idf=False, es decir, TF normalizado pero sin el factor IDF. Otras combinaciones con use_idf=True aparecen también en el top 25 con F1 muy similar. Esto indica que la normalización y la distribución de términos ya son suficientemente informativas, y el aporte del IDF es menor.

Un F1-macro ≈ 0.70 en 20 clases es un desempeño razonable: muy por encima de un clasificador aleatorio o de mayoría y del modelo del punto 2 (F1 score = 0.50).


**Punto 4**

Este punto es prácticamente el mismo que el 1 pero con la matriz transpuesta.

Transponemos la matriz:

In [14]:
X_train_transpuesta = X_train.transpose()

Imprimimos el vocabulario y seleccionamos 5 palabras interesantes:

In [15]:
idx2word

{95844: 'was',
 97181: 'wondering',
 48754: 'if',
 18915: 'anyone',
 68847: 'out',
 88638: 'there',
 30074: 'could',
 37335: 'enlighten',
 60560: 'me',
 68080: 'on',
 88767: 'this',
 25775: 'car',
 80623: 'saw',
 88532: 'the',
 68781: 'other',
 31990: 'day',
 51326: 'it',
 34809: 'door',
 84538: 'sports',
 57390: 'looked',
 89360: 'to',
 21987: 'be',
 41715: 'from',
 55746: 'late',
 9843: '60s',
 35974: 'early',
 11174: '70s',
 25492: 'called',
 24160: 'bricklin',
 34810: 'doors',
 96247: 'were',
 76471: 'really',
 83426: 'small',
 49447: 'in',
 16809: 'addition',
 41724: 'front',
 24635: 'bumper',
 81658: 'separate',
 77878: 'rest',
 67670: 'of',
 23480: 'body',
 51136: 'is',
 17936: 'all',
 54632: 'know',
 25590: 'can',
 88143: 'tellme',
 62746: 'model',
 64931: 'name',
 37287: 'engine',
 84276: 'specs',
 99911: 'years',
 73373: 'production',
 96433: 'where',
 59079: 'made',
 46814: 'history',
 68409: 'or',
 96395: 'whatever',
 49932: 'info',
 100208: 'you',
 45885: 'have',
 41979: '

Seleccionamos estas palabras:

* 'body' (23480)
* 'engine' (37287)
* 'weapons' (96087)
* 'morality' (63096)
* 'microgravity' (61579)

In [16]:
palabras_idx = [23480, 37287, 96087, 63096, 61579]

Analizamos las palabras seleccionadas:

In [21]:
def _analizar_palabras_similares(palabras_idx):
  # Recorremos las 5 palabras seleccionadas
  for idx_1, palabra_idx_1 in enumerate(palabras_idx):

    print("-" * 120)
    print("-" * 120)
    print(f"Palabra número {idx_1 + 1}")
    print("-" * 16)
    print(f"Palabra: {idx2word[palabra_idx_1]}")
    print(f"Índice: {palabra_idx_1}")
    print("-" * 100)

    # Calculamos la similaridad coseno con todas las palabras de train transpuesta
    cossim = cosine_similarity(X_train_transpuesta[palabra_idx_1], X_train_transpuesta)[0]
    # Ordenamos por similitud, salteamos la primera (que es la misma palabra) y tomamos las siguientes 5 para analizar
    # (es decir, las 5 más similares) (Podría ser que la primera sea otra palabra si coincide en estar exactamente en
    # los mismos documentos, pero descartamos ese caso por ser muy improbable.)
    for idx_2, palabra_idx_2 in enumerate(np.argsort(cossim)[::-1][1:6]):

      print("     " + "-" * 80)
      print(f"     Comparación {idx_2 + 1}")
      print("     " + "-" * 13)
      print(f"     Palabra: {idx2word[palabra_idx_2]}")
      print(f"     Índice: {palabra_idx_2}")
      print(f"     Similaradidad coseno: {cossim[palabra_idx_2]}")
      print("     " + "-" * 80)
      
    print("-" * 120)
    print("-" * 120)
    print("\n" * 3)



_analizar_palabras_similares(palabras_idx)

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
Palabra número 1
----------------
Palabra: body
Índice: 23480
----------------------------------------------------------------------------------------------------
     --------------------------------------------------------------------------------
     Comparación 1
     -------------
     Palabra: constantinopolitan
     Índice: 29430
     Similaradidad coseno: 0.2580608307859134
     --------------------------------------------------------------------------------
     --------------------------------------------------------------------------------
     Comparación 2
     -------------
     Palabra: supplement
     Índice: 86511
     Similaradidad coseno: 0.25144935616550085
     -------------------------------------------------------------------

**Análisis**

<br />

1. "body"

Vecinos: "constantinopolitan", "supplement", "akins", "corrado", "resurrection" (similaridades coseno ≈ 0.21–0.26). 

Hay algo de sentido temático en que aparezca "resurrection" y "constantinopolitan "(contexto religioso / teológico), pero otras como "akins" o "corrado" parecen más bien apellidos o términos muy específicos. Esto sugiere que, para palabras relativamente frecuentes y con diferentes significados como "body", los vecinos mezclan algo de significado de las palabras con ruido generado por co-ocurrencias puntuales.

<br />

2. "engine"

Vecinos: "guesser", "tripmeter", "mountings", "rebuilt", "carburator", con similaridades coseno muy cercanas (≈ 0.20). 

Acá la interpretación es mucho más clara: "tripmeter", "mountings", "rebuilt", "carburator" son todos términos claramente mecánicos / automotrices. Indica que el modelo está capturando bien el contexto de "engine" dentro de documentos sobre autos y reparaciones.

<br />

3. "weapons"

Vecinos: "sarin", "weapon", "millitary", "facist", "azerbadjan", con similaridades coseno ~0.23–0.28. 

La lista es bastante coherente:
* "sarin" es un agente químico usado como arma.
* "weapon" es el singular de la misma palabra.
* "millitary" (typo de "military"), "facist", "azerbadjan" se asocian a contextos de guerras, terrorismo, etc.

En este caso, la matriz término–documento refleja bien la cercanía de temas relacionados a guerra, violencia y armamento.

<br />

4. "morality"

Vecinos: "objective", "moral", "deduce", "subjectivity", "analysing", con similaridades coseno ~0.25–0.46.

La relación de sentido es muy clara:

* "moral", "objective", "subjectivity" tiene que ver con debates sobre ética.

* "deduce" y "analysing" tiene que ver con argumentativos o filosóficos.

La similaridad coseno está agrupando términos típicos de discusión filosófica sobre ética.

<br />

5. "microgravity"

Vecinos: "centralize", "retrofitted", "sif", "explified", "constellations", todos con una similaridades coseno altas (≈ 0.96). 

"microgravity" es un término muy específico del dominio espacial.

Algunos vecinos ("centralize", "retrofitted", "sif", "explified") no están claramente relacionados semánticamente; parecen términos raros o poco frecuentes, tal vez incluso con errores ortográficos.

Las similitudes tan altas y todas prácticamente iguales sugieren que estas palabras aparecen casi siempre en un mismo grupo reducido de documentos. Co-ocurrencias raras pueden generar "vecinos" dificiles de interpretar.

<br />

Conclusión:

La matriz término–documento permite capturar familias de palabras coherentes en torno a un tema, pero también muestra limitaciones habituales de los modelos basados sólo en co-ocurrencias, sobre todo con vocabularios grandes y términos poco frecuentes.